In [1]:
from src.database_manager import SQLTableManager
import requests
from bs4 import BeautifulSoup
import re
from io import StringIO
import time
from datetime import datetime

from typing import Union

import pandas as pd
pd.set_option('display.max_rows', None)

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from src.custom_logger import CustomLogger

#### Initiliazing the Logger

In [2]:
# Create an instance of CustomLogger with logger name and log directory
logger_instance = CustomLogger("scraper","scraper",r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\Python\Scraping\RRC\src\logs")

# Get the logger
logger = logger_instance.get_logger()

# Start the logger
logger.info(f"################## Logging Started ##################")

[scraper] INFO (05-10 12:31 PM): ################## Logging Started ################## (Line: 8) [865591240.py]



#### Defining URLs

In [3]:
# Defining URL's
base_url = 'https://webapps2.rrc.texas.gov/EWA/'
query_url = base_url + 'drillingPermitsQueryAction.do'

#### Defining Functions

In [4]:
def convert_dt(date_str:str) -> datetime.date:
    """
    date_str: YYYY-MM-DD
    """
    try:
        return datetime.strptime(date_str, '%Y-%m-%d').date()
    except ValueError:
       logger.error(f"Incorrect data format {date_str}. Should be YYYY-MM-DD")

       
def get_APIs_from_SQL(date:datetime.date) -> pd.DataFrame:

    # Create an instance of the class
    sql_connector = SQLTableManager()

    try:
        sql_connector.connect()
        
        query = f'''SELECT distinct [API_10] FROM [Sitio_GIS].[dbo].[STR_WELL_UNITS]
                    where ([API_10] like '42%') and ([Sitio_GIS].[dbo].[STR_WELL_UNITS].[created_date] >= '{date}');'''
        
        df = sql_connector.execute_query(sql_query=query)

        logger.info(f"Total API Count: {df.shape[0]}")

        logger.debug(
            f"API DataFrame Top 5 Rows:\n {df.head()}\n----------\n")

        return df

    except Exception as e:
        logger.error(f"Error occured while reading {query} from server MsSQL server. Error details:",exc_info=True)

    finally:
        sql_connector.close_connection()


def get_headers() -> dict:

    headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Host': 'webapps2.rrc.texas.gov',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'sec-ch-ua': "Google Chrome;v='119', 'Chromium';v='119', 'Not?A_Brand';v='24'",
    'sec-ch-ua-mobile': "?0",
    'sec-ch-ua-platform': "Windows"
    }
    
    return headers


def payload(api:str) -> dict:
         
    """      
    Parameters
    ----------
    api : TYPE, int
        DESCRIPTION. 10 digit API. Strips API of initial '42'.

    Returns
    -------
    Dict.
    """

    # Check the length of API

    try:
        assert len(str(api)) == 14 or len(str(api)) == 10, f"Length of API: {api} don't match with 10 or 14 digit"
        
        if len(str(api)) == 14:
            api = (str(api)[2:10])
        
        elif len(str(api)) == 10:
            api = (str(api)[2:])

    except AssertionError as e:
        logger.error(e)

    payload = {
    'methodToCall': 'search',
    'searchArgs.apiNoHndlr.inputValue': api,
    'searchArgs.operatorNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.leaseNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.fieldNameWildcardHndlr.inputValue': 'beginsWith',
    'searchArgs.surveyNameWildcardHndlr.inputValue': 'beginsWith'
    }

    return payload


def get_data_DrillingPermitQuery(req_url:str, headers:dict, payload:dict) -> requests.Response:
    """      
    Parameters
    ----------
    post_response : TYPE, Requests Response
        DESCRIPTION. .

    Returns
    -------
    Request Response.
    """
    try:
        with requests.session() as s:
            s.verify = False
            res_get = s.get(url=req_url, headers=headers, verify=False)

            if res_get.status_code == 200:
                cookies = res_get.cookies
                res_post = s.post(url=query_url, data=payload, cookies=cookies, headers=headers, verify=False)

                return res_post

    except Exception as e:
        logger.error(f"Error occured while requesting {req_url}. Error details:",exc_info=True)


def parse_query_data(post_resp:requests.Response) -> tuple[pd.DataFrame,pd.DataFrame,str,datetime.date]:
    """      
    Parameters
    ----------
    post_response : TYPE, Requests Response
        DESCRIPTION. Response from the get_data_DrillingPermitQuery()

    Returns
    -------
    DataFrame.
    """
    soup = BeautifulSoup(post_resp.content,'html.parser')
    web_links = soup.select('a')
    href = [base_url + web_link['href'] for web_link in web_links if 'drillingPermitDetailAction' in web_link['href']]

    df_main = pd.read_html(StringIO(post_resp.text))[9]
    df_main.columns = df_main.iloc[1,:]
    df_main = df_main.drop([0,1]).copy()
    df_main = df_main.reset_index(drop=True).copy()

    df_edit = df_main.copy()
    df_edit['API NO.'] = df_main['API NO.'].str.split()[0][0]
    df_edit['HREF'] = href
    df_edit[['Submitted_Dt', 'Approved_Dt']] = df_edit['Status Date'].str.split(expand=True).loc[:,[1,3]]
    df_edit['Submitted_Dt'] = pd.to_datetime(df_edit['Submitted_Dt'])
    df_edit['Approved_Dt'] = pd.to_datetime(df_edit['Approved_Dt'])

    apprvd_Dt_max_idx = df_edit.index[df_edit['Approved_Dt']==df_edit['Approved_Dt'].max()]
    max_apprvd_Dt = df_edit['Approved_Dt'].max()
    href_link = df_edit.loc[apprvd_Dt_max_idx,'HREF'].item()

    return df_main, df_edit, href_link, max_apprvd_Dt, apprvd_Dt_max_idx.item()


def get_data_W1_Form(url_href:str) -> requests.Response:
    """      
    Parameters
    ----------
    url_href : TYPE, Str
        DESCRIPTION. URL from parse_query_data()

    Returns
    -------
    Response.
    """

    with requests.session() as s:
        s.verify = False
        res_get = s.get(url=url_href,headers=get_headers(),verify=False)
    
    return res_get


def parse_W1_Form(get_resp:requests.Response) -> pd.DataFrame:
    """      
    Parameters
    ----------
    get_resp : TYPE, Requests Response
        DESCRIPTION. get response from get_data_W1_Form()

    Returns
    -------
    DataFrame
    """
    soup = BeautifulSoup(get_resp.text,'html.parser')
    tables = soup.find_all('table',class_='GroupBox1')

    data_dict = {}

    for count,table in enumerate(tables):
        if (table.find_all('th') is not None):
            for i,(th,td) in enumerate(zip(table.find_all('th'),table.find_all('td'))):
                if th.text == 'Horizontal Wellbore':
                    data_dict[th.text] = td.text
                elif th.text == 'Acres':
                    data_dict[th.text] = td.text

    return pd.DataFrame.from_dict(data_dict, orient='index').T


def formatting_results_df(dataframe:pd.DataFrame) -> pd.DataFrame:
    """
    Formatting dfs_unique_Results DataFrame
    """

    # Formatting the columns
    dataframe.columns = dataframe.columns.str.lower().str.replace(".","").str.replace("#","").str.strip().str.replace(" ","_")

    # Keeping relevant columns
    dataframe = dataframe[['api_no', 'lease', 'well_number', 'filing_purpose', 'amend', 
                           'status', 'submitted_dt','approved_dt', 'horizontal_wellbore', 'acres','href']].copy()
    return dataframe


def results_to_db(dataframe:pd.DataFrame, sqlTableName:str='well_search_results') -> None:

    manager = SQLTableManager()

    try:
        manager.connect(dbname='Sandbox_Engineering')
        manager.add_rows_from_dataframe(sql_table_name=sqlTableName, dataframe=dataframe)
        logger.info(f"Added rows to {sqlTableName} \n{dataframe}")
    
    except Exception as e:
        
        # If SQL insert fails, create csv file in the working directory
        now = datetime.now().strftime('%m_%d_%Y-%H_%M_%S')
        filename: str = f'RRC_Scraper_Results_{now}.csv'
        dataframe.to_csv(filename, index=False)
        
        logger.error(f"Failed to add rows to {sqlTableName}. Created the {filename} file. Error details:",exc_info=True)
    finally:
        manager.close_connection()

In [5]:
def main(dt:str) -> None:
    """
    dt: Date Input Format YYYY-MM-DD
    """

    # Get API's from STR_WELL_UNITS DB >= Date
    date_filter = convert_dt(dt)

    # Running Query and getting the API's into DataFrame df_api
    df_api = get_APIs_from_SQL(date_filter)
    
    logger.info(f"Running the scraper on {df_api.shape[0]} unique wells.")

    # Defining empty lists
    dfs_main = []
    dfs_edit = []
    dfs_unique_Results = []

    # Define Headers
    headers = get_headers()

    # Running the loop
    for idx, api in enumerate(df_api['API_10']):

        payload_api = payload(api=api)

        if idx > 1:
            time.sleep(0.5)

        response_from_DrillingPermit_Query = get_data_DrillingPermitQuery(req_url=query_url, headers=headers, payload=payload_api)

        df_mainPage, df_edit, w1_link, Approved_Dt_max, idx_Max_Aprvd_Dt = parse_query_data(post_resp = response_from_DrillingPermit_Query)

        if idx > 1:
            time.sleep(0.5)

        response_from_W1_query = get_data_W1_Form(url_href=w1_link)

        df_w1 = parse_W1_Form(get_resp=response_from_W1_query)
        df_w1['Approved_Dt'] = Approved_Dt_max

        df_edit_merge_with_w1 = df_edit.merge(df_w1,on='Approved_Dt',how='left').reset_index(drop=True)

        df_unique_res = df_edit_merge_with_w1[df_edit_merge_with_w1['Approved_Dt']==Approved_Dt_max].reset_index(drop=True).copy()

        dfs_edit.append(df_edit_merge_with_w1)
        dfs_main.append(df_mainPage)
        dfs_unique_Results.append(df_unique_res)

        time.sleep(0.5)

        if idx==5:
            break

    logger.info(f"All {df_api.shape[0]} wells got succesfully scraped!")


    # Formating unique results dataframe's columns
    df_Results = formatting_results_df(dataframe=pd.concat(dfs_unique_Results).reset_index(drop=True))

    # Pushing data to SQL
    logger.info("Pushing scraper results to SQL!")
    results_to_db(dataframe=df_Results)

##### Main

In [6]:
if __name__ == "__main__":

    # Running Main Program (Change the date from where you want the grab the APIs from)
    main('2024-04-15')

c:\users\apoorva.saxena\onedrive - sitio royalties\desktop\project - apoorva\python\scraping\rrc\src\database_manager.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(sql_query, self.connection)
[scraper] INFO (05-10 12:31 PM): Total API Count: 130 (Line: 24) [2753276606.py]

[scraper] INFO (05-10 12:31 PM): Running the scraper on 130 unique wells. (Line: 12) [2910409013.py]

[scraper] INFO (05-10 12:31 PM): All 130 wells got succesfully scraped! (Line: 55) [2910409013.py]

[scraper] INFO (05-10 12:31 PM): Pushing scraper results to SQL! (Line: 62) [2910409013.py]

[scraper] INFO (05-10 12:31 PM): Added rows to well_search_results 
     api_no                 lease well_number filing_purpose amend    status  \
0  12335385             WILLEKE A         21H      New Drill     N  APPROVED   
1  12335388    